# Создание датасета и вопросов для RAG из файлов с сайта

In [2]:
import json
import random
from openai import OpenAI

from tqdm import tqdm


from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders.text import TextLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

In [3]:
loader = DirectoryLoader("../data/from_site/md", glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding':'utf-8'})
docs = loader.load()
len(docs)

26

In [ ]:
loader = DirectoryLoader("../data/from_site/md/faq", glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding':'utf-8'})
docs = loader.load()
len(docs)

In [3]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)

In [ ]:
docs[0].dict()

In [5]:
chunks = []

for doc in docs:
    metadata = doc.metadata
    page_content = doc.page_content
    doc_chunks = markdown_splitter.split_text(page_content)
    for chunk in doc_chunks:
        chunk.metadata = chunk.metadata | metadata
        chunks.append(chunk)

In [ ]:
len(chunks)

In [7]:
client = OpenAI()
def complete(query: str, instruction: str):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": instruction},
            {
                "role": "user",
                "content": query
            }
        ]
    )
    return completion

def extract_content(completion) -> str:
    try:
        return completion.choices[0].message.content
    except:
        return '==ERROR=='

In [8]:
def create_chunk_from_document(doc) -> str:
    headers = '\n'.join([doc.metadata[meta] for meta in doc.metadata if meta.startswith('Header')])
    page_content = doc.page_content
    result = headers + '\n' + page_content
    return result

## Генерация вопроса на 1 чанк

In [9]:
SYSTEM_PROMPT = "Тебе прислали кусок текста из FAQ, придумай вопрос, ответ на который требовал бы информации из этого куска. Вопросы должны относиться с сервису RUTUBE. И быть написаны, будто бы они идут от клиента специалисту службы поддержки. Вопрос должен быть один."

In [10]:
def create_1_chunk_instruction(chunk:str) -> str:
    result = f'КУСОК ТЕКСТА ИЗ FAQ\n=====\n{chunk}\n=====\n'
    return result

In [11]:
def generate_answers_single_chunk(documents_list:list[str]) -> list[dict]:
    result = []
    for doc in tqdm(documents_list):
        sub_result = {}
        chunk = create_1_chunk_instruction(create_chunk_from_document(doc))
        question_completion = complete(chunk, SYSTEM_PROMPT)
        question = extract_content(question_completion)
        sub_result['question'] = question
        sub_result['chunk'] = chunk
        sub_result['metadata'] = {'answer': question_completion.to_dict()} | {'doc':doc.dict()}
        result.append(sub_result)

    return result

In [ ]:
result = generate_answers_single_chunk(chunks)

In [ ]:
result

In [ ]:
with open('synth_complex_questions_1_chunk.json', 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

print("Данные успешно сохранены")

## Генерация вопроса на 2 чанка

In [14]:
def create_chunk_pair(chunk_0:str, chunk_1:str) -> str:
    result = f'''КУСОК 1:\n=====\n{chunk_0}\n=====\n\nКУСОК 2\n=====\n{chunk_1}\n=====\n'''.strip()
    return result


# SYSTEM_PROMPT = "Тебе прислали 2 куска текста, придумай вопрос, ответ на который требовал бы информации из двух чанков сразу. Вопросы должны относиться с сервису RUTUBE. И быть написаны, будто бы они идут от клиента специалисту службы поддержки."

SYSTEM_PROMPT = "Тебе прислали 2 куска текста, придумай вопрос, ответ на который требовал бы информации из двух чанков сразу. Вопросы должны относиться с сервису RUTUBE. И быть написаны, будто бы они идут от клиента специалисту службы поддержки. Вопрос должен быть один, чтобы ответ на него сочетал информацию из разных кусков."

# SYSTEM_PROMPT = "Тебе прислали 2 куска текста, придумай вопрос, ответ на который требовал бы информации из двух чанков сразу. Вопросы должны относиться с сервису RUTUBE. И быть написаны, будто бы они идут от клиента специалисту службы поддержки. Вопрос должен быть один, чтобы ответ на него сочетал информацию из разных кусков. Вопрос должен быть сформулирован по обоим чанкам при условии одного от другого. Если вопрос куски текста по смыслу не подходят для создания объединенного вопроса, то напиши ПРОПУСК."



In [16]:
def generate_answers_double_chunk(documents_list:list[tuple]) -> list[dict]:
    result = []
    for doc_a, doc_b in tqdm(documents_list):
        sub_result = {}
        chunk_a = create_chunk_from_document(doc_a)
        chunk_b = create_chunk_from_document(doc_b)
        chunk_pair = create_chunk_pair(chunk_a,chunk_b)
        question_completion = complete(chunk_pair)
        question = extract_content(question_completion)
        sub_result['question'] = question
        sub_result['chunk_a'] = chunk_a
        sub_result['chunk_b'] = chunk_b
        sub_result['metadata'] = {'answer': question_completion.to_dict()} | {'doc_a':doc_a.dict()} | {'doc_b': doc_b.dict()}
        result.append(sub_result)

    return result
        


In [17]:
def get_random_pairs(lst, num_pairs):
    all_pairs = [(a, b) for i, a in enumerate(lst) for b in lst[i+1:]]
    
    if num_pairs > len(all_pairs):
        raise ValueError("Количество пар больше, чем возможные уникальные комбинации.")
    
    return random.sample(all_pairs, num_pairs)


In [ ]:
chunks[0]

In [ ]:
result = generate_answers_double_chunk(get_random_pairs(chunks, 100))

In [ ]:
result

In [ ]:
with open('synth_complex_questions_2_chunks.json', 'w', encoding='utf-8') as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

print("Данные успешно сохранены")
